# Scraping Rayban

In [2]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd


from time import sleep

# Importar librerías para automatización de navegadores web con Selenium
# -----------------------------------------------------------------------
from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select  # Select se utiliza para interactuar con elementos <select> en páginas web.
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException # Excepciones comunes de selenium que nos podemos encontrar 
from selenium.webdriver.common.by import By


import seaborn as sns
import plotly.express as px  # type: ignore
import plotly.graph_objects as go # type: ignore
from plotly.subplots import make_subplots # type: ignore
import warnings
warnings.filterwarnings('ignore')



In [34]:
driver = webdriver.Chrome()
url_parafina = "https://www.ray-ban.com/spain/gafas-de-sol/ver-todas"
driver.get(url_parafina)
driver.maximize_window()
sleep(6)

# Cajamos un poco para que detecte el ratón
driver.execute_script("window.scrollTo(0, 1000);")
sleep(3)

# Cerramos el anuncio
driver.find_element("css selector", "#rb-header-app > div.modal-wrapper.modal-wrapper--header.modal-wrapper--display > div.modal-content-wrapper > div > button").click()
sleep(2)

# Cerramos las cookies
driver.find_element("css selector", "#consent_prompt_submitNo").click()
sleep(2)


# Bajamos hasta abajo para cargar más productos
driver.execute_script("window.scrollTo(0, 5500);")
sleep(4)

# pinchamos para cargar más
driver.find_element("css selector", "body > div.rb-app__main.static-header.loaded.rb-app__header--static > div > div.rb-load-more > button").click()
sleep(2)

for i in range(20):

    try:
    # Bajamos hasta abajo para cargar más productos
        driver.execute_script("window.scrollTo(0, 6000);")
        sleep(4)

        # pinchamos para cargar más
        driver.find_element("css selector", "body > div.rb-app__main.static-header.loaded.rb-app__header--static > div > div.rb-load-more > button").click()
        sleep(2)
        
    except:
        break


html_table_page = driver.page_source



In [48]:
sopa = BeautifulSoup(html_table_page, "html.parser")
elementos_gafas = sopa.find("div", {"class":"rb-products grid"})
lista_gafas = elementos_gafas.findAll("div", {"class":"rb-product-tile"})

dic_gafas = {
    "titulo":[],
    "precio":[]
}

for gafa in lista_gafas:
    div_titulo = gafa.find("div", {"class":"rb-product-tile__content"})
    
    titulo = div_titulo.find("h2").text.strip()
    precio = div_titulo.find("div", {"class":"rb-product-tile__price-container"}).text.split()[1].replace(",",".")
    
    dic_gafas["titulo"].append(titulo)
    dic_gafas["precio"].append(precio)
    


In [54]:
df_rayban = pd.DataFrame(dic_gafas)
df_rayban["precio"] = df_rayban["precio"].astype(float)
df_rayban.to_csv("../datos/df_rayban.csv")

In [55]:
df_rayban.head()

,titulo,precio
0,RB4441D BIO-BASED,137.0
1,ORIGINAL WAYFARER CLASSIC,163.0
2,MEGA BALORAMA,173.0
3,NEW WAYFARER CLASSIC,202.0
4,AVIATOR CLASSIC,163.0


In [56]:
df_rayban.describe().T

,count,mean,std,min,25%,50%,75%,max
precio,483.0,195.819876,56.337221,125.0,163.0,175.0,223.0,548.0
